In [5]:
!conda create -n pycaret_env python=3.8 -y
!conda activate pycaret_env
!conda install -c conda-forge pycaret -y


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.11.1
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0



## Package Plan ##

  environment location: /opt/anaconda3/envs/pycaret_env

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.3.11  |       hecd8cb5_0         128 KB
    libffi-3.4.4               |       hecd8cb5_0         132 KB
    ncurses-6.4                |       hcec6c5f_0        1018 KB
    openssl-3.0.13             |       hca72f7f_0         4.5 MB
    pip-23.3.1                 |   py38hecd8cb5_0         2.6 MB
    python-3.8.19              |       h5ee71fb_0        12.7 MB
    readline-8.2              

In [6]:
!conda install -n pycaret_env ipykernel -y
!python -m ipykernel install --user --name pycaret_env --display-name "Python (PyCaret)"


EnvironmentLocationNotFound: Not a conda environment: /opt/anaconda3/envs/pycaret_env

Installed kernelspec pycaret_env in /Users/rafael/Library/Jupyter/kernels/pycaret_env


In [30]:
import pycaret
print(pycaret.__version__)

3.3.0


In [32]:
from pycaret.classification import setup, compare_models

In [31]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn import model_selection, tree, linear_model, metrics, feature_selection
from sklearn.model_selection import validation_curve
import joblib
import logging
from sklearn.metrics import log_loss, f1_score, roc_curve
from mlflow.models.signature import infer_signature
from mlflow.tracking import MlflowClient

In [2]:
!pip install mlflow

In [1]:
import mlflow

In [6]:
conda install -c conda-forge pycaret

WARNING conda.core.index:push_record(251): Skipping conda-forge/osx-64::sktime-0.28.0-py311h6eed73b_0 due to InvalidSpec: >=0.4,<0.4.4,
WARNING conda.core.index:push_record(251): Skipping conda-forge/osx-64::sktime-0.28.0-py38h50d1736_0 due to InvalidSpec: >=0.4,<0.4.4,
WARNING conda.core.index:push_record(251): Skipping conda-forge/osx-64::sktime-0.28.0-py39h6e9494a_0 due to InvalidSpec: >=0.4,<0.4.4,
- WARNING conda.models.version:get_matcher(544): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.7.1.*, but conda is ignoring the .* and treating it as 1.7.1
done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

InvalidVersionSpec: Invalid version '>=0.4,<0.4.4,': cannot join single expression


Note: you may need to restart the kernel to use updated packages.


In [34]:
file_path = '/Users/rafael/Desktop/KOBE/kobedata.csv'

df = pd.read_csv(file_path)
print(df.head())

         action_type combined_shot_type  game_event_id   game_id      lat  \
0          Jump Shot          Jump Shot             10  20000012  33.9723   
1          Jump Shot          Jump Shot             12  20000012  34.0443   
2          Jump Shot          Jump Shot             35  20000012  33.9093   
3          Jump Shot          Jump Shot             43  20000012  33.8693   
4  Driving Dunk Shot               Dunk            155  20000012  34.0443   

   loc_x  loc_y       lon  minutes_remaining  period  ...       shot_type  \
0    167     72 -118.1028                 10       1  ...  2PT Field Goal   
1   -157      0 -118.4268                 10       1  ...  2PT Field Goal   
2   -101    135 -118.3708                  7       1  ...  2PT Field Goal   
3    138    175 -118.1318                  6       1  ...  2PT Field Goal   
4      0      0 -118.2698                  6       2  ...  2PT Field Goal   

          shot_zone_area  shot_zone_basic  shot_zone_range     team_id  \


## Configurar o MLFlow

In [35]:
# Para usar o sqlite como repositório
mlflow.set_tracking_uri('sqlite:///mlruns.db')

experiment_name = 'Projeto_Kobe'
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment(experiment_id)
experiment_id = experiment.experiment_id


In [12]:
train_test_split?

Object `train_test_split` not found.


## Preparação de Dados

In [36]:
df_dev = pd.read_parquet('/Users/rafael/Desktop/KOBE/dataset_kobe_dev.parquet')

In [37]:
df_dev.columns

Index(['action_type', 'combined_shot_type', 'game_event_id', 'game_id', 'lat',
       'loc_x', 'loc_y', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'season', 'seconds_remaining', 'shot_distance', 'shot_made_flag',
       'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
       'team_id', 'team_name', 'game_date', 'matchup', 'opponent', 'shot_id'],
      dtype='object')

In [38]:
print(df_dev.head())

         action_type combined_shot_type  game_event_id   game_id      lat  \
0          Jump Shot          Jump Shot             10  20000012  33.9723   
1          Jump Shot          Jump Shot             12  20000012  34.0443   
2          Jump Shot          Jump Shot             35  20000012  33.9093   
3          Jump Shot          Jump Shot             43  20000012  33.8693   
4  Driving Dunk Shot               Dunk            155  20000012  34.0443   

   loc_x  loc_y       lon  minutes_remaining  period  ...       shot_type  \
0    167     72 -118.1028                 10       1  ...  2PT Field Goal   
1   -157      0 -118.4268                 10       1  ...  2PT Field Goal   
2   -101    135 -118.3708                  7       1  ...  2PT Field Goal   
3    138    175 -118.1318                  6       1  ...  2PT Field Goal   
4      0      0 -118.2698                  6       2  ...  2PT Field Goal   

          shot_zone_area  shot_zone_basic  shot_zone_range     team_id  \


In [39]:
# Contar linhas vazias
linhas_vazias = df_dev.isna().all(axis=1).sum()

print(f"Existem {linhas_vazias} linhas completamente vazias no dataset.")

Existem 0 linhas completamente vazias no dataset.


In [40]:
# Contar valores nulos em cada coluna
valores_nulos = df_dev.isna().sum()

print(valores_nulos)

action_type              0
combined_shot_type       0
game_event_id            0
game_id                  0
lat                      0
loc_x                    0
loc_y                    0
lon                      0
minutes_remaining        0
period                   0
playoffs                 0
season                   0
seconds_remaining        0
shot_distance            0
shot_made_flag        3986
shot_type                0
shot_zone_area           0
shot_zone_basic          0
shot_zone_range          0
team_id                  0
team_name                0
game_date                0
matchup                  0
opponent                 0
shot_id                  0
dtype: int64


In [41]:
# O código acima mostra que a coluna shot_made_flag do DataFrame df_dev contém 3986 valores nulos.
# Abaixo mostramos o código que realiza uma limpeza no DataFrame, nesse caso excluindo a coluna.

df_dev_clean = df_dev.dropna(subset=['shot_made_flag'])
y = df_dev_clean['shot_made_flag']

print(f"Antes da limpeza: {df_dev.shape}")
print(f"Depois da limpeza: {df_dev_clean.shape}")


Antes da limpeza: (24271, 25)
Depois da limpeza: (20285, 25)


In [42]:
# Salvando o DataFrame limpo em um arquivo .parquet
df_dev_clean.to_parquet('df_dev_clean.parquet')

In [43]:
# Confirmando que o arquivo foi salvo corretamente e contém os dados esperados.
df_loaded = pd.read_parquet('df_dev_clean.parquet')
print(df_loaded.head())

         action_type combined_shot_type  game_event_id   game_id      lat  \
1          Jump Shot          Jump Shot             12  20000012  34.0443   
2          Jump Shot          Jump Shot             35  20000012  33.9093   
3          Jump Shot          Jump Shot             43  20000012  33.8693   
4  Driving Dunk Shot               Dunk            155  20000012  34.0443   
5          Jump Shot          Jump Shot            244  20000012  34.0553   

   loc_x  loc_y       lon  minutes_remaining  period  ...       shot_type  \
1   -157      0 -118.4268                 10       1  ...  2PT Field Goal   
2   -101    135 -118.3708                  7       1  ...  2PT Field Goal   
3    138    175 -118.1318                  6       1  ...  2PT Field Goal   
4      0      0 -118.2698                  6       2  ...  2PT Field Goal   
5   -145    -11 -118.4148                  9       3  ...  2PT Field Goal   

          shot_zone_area  shot_zone_basic  shot_zone_range     team_id  \


In [44]:
# Contar valores nulos em cada coluna
valores_nulos = df_dev_clean.isna().sum()

print(valores_nulos)

action_type           0
combined_shot_type    0
game_event_id         0
game_id               0
lat                   0
loc_x                 0
loc_y                 0
lon                   0
minutes_remaining     0
period                0
playoffs              0
season                0
seconds_remaining     0
shot_distance         0
shot_made_flag        0
shot_type             0
shot_zone_area        0
shot_zone_basic       0
shot_zone_range       0
team_id               0
team_name             0
game_date             0
matchup               0
opponent              0
shot_id               0
dtype: int64


## Separando a Base de Dados entre Treino e Teste, com descrição abaixo:


Dividir dados em conjuntos de treino e teste é crucial para avaliar corretamente como um modelo de
machine learning generaliza para novos dados, ajudando a evitar o overfitting, ou seja,
quando um modelo aprende tão bem os dados de treino que se torna incapaz de fazer previsões
precisas sobre novos dados. Essa separação permite uma avaliação imparcial do modelo.

Para minimizar vieses e melhorar a generalização, algumas estratégias importantes incluem:

Garantir diversidade e representatividade nos dados.
Usar estratificação para manter proporções de classes consistentes entre treino e teste.
Aplicar validação cruzada para testar a robustez do modelo em diferentes segmentos dos dados.
Em determinados casos, a augmentação de dados pode gerar mais variações para treinamento,
ajudando o modelo a aprender padrões mais generalizáveis.
Essencialmente, essas práticas visam construir modelos mais robustos, justos e capazes de funcionar
bem em situações do mundo real.

In [4]:
# Encerra qualquer execução ativa para rodar o novo run do código abaixo
mlflow.end_run()

NameError: name 'mlflow' is not defined

In [3]:
# Separando base de dados treino e teste e Run Ml Flow

from sklearn.model_selection import train_test_split

data_cols = ['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance']
train_perc = 0.8

with mlflow.start_run(experiment_id=experiment_id, run_name = 'PreparacaoDados'):

    df_dev_clean = pd.read_parquet('/Users/rafael/Desktop/KOBE/df_dev_clean.parquet')
    df_dev_clean = df_dev_clean[data_cols].copy()

    #df_dev_clean.to_parquet('/content/data_filtered.parquet')

    xtrain, xtest, ytrain, ytest = train_test_split(df_dev_clean[['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance']],
                                                  y,
                                                  train_size=train_perc,
                                                  stratify=y)
    # Abaixo vamos mostrar o shape do treino e teste:

    print(f'xtrain shape: {xtrain.shape}')
    print(f'xtest shape: {xtest.shape}')
    print(f'ytrain shape: {ytrain.shape}')
    print(f'ytest shape: {ytest.shape}')

    # Importando o DataFrame com a variável alvo inclusa na base de dados.
    xtrain['shot_made_flag'] = ytrain
    xtest['shot_made_flag'] = ytest

    xtrain.to_parquet('base_train_kobe.parquet')
    xtest.to_parquet('base_test_kobe.parquet')

    # Passando parâmetros MLFlow

    mlflow.log_params({
        'perc-teste': 1-train_perc,
        'colunas-selecionadas': data_cols
    })
    mlflow.log_metrics({
        'qtd_linhas_treino': xtrain.shape[0],
        'qtd_linhas_teste': xtest.shape[0],
    })


NameError: name 'mlflow' is not defined

## Treinamento do Modelo

In [2]:
list_models = exp.compare_models(['lr', 'dt'], n_select=2, sort='f1')
print(list_models)  # Veja se algum modelo foi retornado


NameError: name 'exp' is not defined

In [ ]:
!pip install --upgrade jupyterlab jupyterlab_widgets ipywidgets

In [6]:
print(xtrain.head())
print(xtrain.describe())
print(xtrain['shot_made_flag'].value_counts())


NameError: name 'xtrain' is not defined

In [3]:
# Instalar MLflow
!pip install mlflow

In [2]:
!pip install "pycaret[full]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 3.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 8.3 MB/s eta 0:00:0000:0100:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fugue to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

In [5]:
!pip install --ignore-installed mlflow

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached PyYAML-6.0.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata (2.1 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached importlib_metadata-7.1.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached werkzeug-3.0.2-py3-none-any.whl.metadata (4.1 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl.metadata (2.9 kB)
  Using cached blinker-1.7.

In [10]:
pip install "matplotlib<3.8.0" "pandas<2.2.0" "scipy<=1.11.4"

Note: you may need to restart the kernel to use updated packages.


In [11]:
# DIGITADO AQUI
import os
from sklearn.metrics import log_loss, f1_score
import mlflow
from pycaret.classification import *

registered_model_name = 'modelo_kobe'
nexamples = 5
model_version = -1
experiment_id = mlflow.create_experiment('Treinamento Kobe Atualizado')

with mlflow.start_run(experiment_id=experiment_id, run_name = 'Treinamento'):

    exp = pc.setup(
            data = xtrain,
            target = 'shot_made_flag',
            test_data = xtest,
            normalize=True,
            log_experiment = False
    )
    list_models = exp.compare_models(['lr', 'dt'], n_select=2, sort='f1')

    #ARVORE

    #exp.plot_model(list_models[0], plot='vc', save = True)
    yhat_test = exp.predict_model(list_models[0])
    plot_parameter_validation_curve(xtrain.drop('shot_made_flag', axis=1), ytrain, 'max_depth', {'max_depth': [2, 3, 4, 5, 6, 7, 8]},
                                        list_models[0], 'Árvore de Decisão', 'f1',
                                        logx=False)
    plt.savefig('dt_validation_curve.png')
    mlflow.log_artifact('dt_validation_curve.png')
    #os.remove('dt_validation_curve.png')

    mlflow.log_metrics({
        'dt_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'dt_f1': f1_score(yhat_test.target, yhat_test.prediction_label),
    })

    #REGRESSAO

    yhat_test = exp.predict_model(list_models[1])
    plot_parameter_validation_curve(xtrain.drop('shot_made_flag', axis=1), ytrain, 'C', {'C': [0.001, 0.01, 0.1, 1, 10]},
                                        list_models[1], 'Regressão Logística', 'f1',
                                        logx=True)
    
    plt.savefig('lr_validation_curve.png')
    mlflow.log_artifact('lr_validation_curve.png')
    #os.remove('dt_validation_curve.png')

    mlflow.log_metrics({
        'lr_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'lr_f1': f1_score(yhat_test.target, yhat_test.prediction_label),
    })


    # FINALIZACAO MELHOR MODELO

    tune_model = exp.tune_model(list_models[0],
                                optimize = 'f1',
                                search_algorithm = 'random',
                                n_iter = 4)
    yhat_test = exp.predict_model(tune_model, raw_score=True)
    mlflow.log_metrics({
        'final_model_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'final_model_f1': f1_score(yhat_test.target, yhat_test.prediction_label),
    })
    
    yhat_test.to_parquet('prediction_test.parquet')
    mlflow.log_artifact('prediction_test.parquet')
    final_model = exp.finalize_model(tune_model)

    # EXPORTACAO PARA LOG E REGISTRO DO MODELO
    exp.save_model(final_model, f'./{registered_model_name}')
    # Carrega novamente o pipeline + bestmodel
    model_pipe = exp.load_model(f'./{registered_model_name}')
    # Assinatura do Modelo Inferida pelo MLFlow
    model_features = list(xtrain.drop('shot_made_flag', axis=1).columns)
    inf_signature = infer_signature(xtrain[model_features],
                                    model_pipe.predict_proba(xtrain.drop('shot_made_flag', axis=1)))
    # Exemplo de entrada para o MLmodel
    input_example = {x: xtrain[x].values[:nexamples] for x in model_features}
    # Log do pipeline de modelagem do sklearn e registrar como uma nova versao
    mlflow.sklearn.log_model(
        sk_model=model_pipe,
        artifact_path="sklearn-model",
        registered_model_name=registered_model_name,
        signature = inf_signature,
        input_example = input_example,
        pyfunc_predict_fn='predict_proba'
    )
     # Criacao do cliente do servico MLFlow e atualizacao versao modelo
    client = MlflowClient()
    if model_version == -1:
        model_version = client.get_latest_versions(registered_model_name)[-1].version
    # Registrar o modelo como staging
    client.set_registered_model_alias(
        name    = registered_model_name,
        alias   = "staging",
        version = model_version
    )


ModuleNotFoundError: No module named 'mlflow'

In [ ]:
# DIGITADO AQUI
import os
from sklearn.metrics import log_loss, f1_score
import mlflow
from pycaret.classification import *

registered_model_name = 'modelo_kobe'
nexamples = 5
model_version = -1
experiment_id = mlflow.create_experiment('Treinamento Kobe Atualizado')

with mlflow.start_run(experiment_id=experiment_id, run_name = 'Treinamento'):

    exp = pc.setup(
            data = xtrain,
            target = 'shot_made_flag',
            test_data = xtest,
            normalize=True,
            log_experiment = False
    )
    list_models = exp.compare_models(['lr', 'dt'], n_select=2, sort='f1')

    #ARVORE

    #exp.plot_model(list_models[0], plot='vc', save = True)
    yhat_test = exp.predict_model(list_models[0])
    plot_parameter_validation_curve(xtrain.drop('shot_made_flag', axis=1), ytrain, 'max_depth', {'max_depth': [2, 3, 4, 5, 6, 7, 8]},
                                        list_models[0], 'Árvore de Decisão', 'f1',
                                        logx=False)
    plt.savefig('dt_validation_curve.png')
    mlflow.log_artifact('dt_validation_curve.png')
    #os.remove('dt_validation_curve.png')

    mlflow.log_metrics({
        'dt_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'dt_f1': f1_score(yhat_test.target, yhat_test.prediction_label),
    })

    #REGRESSAO

    yhat_test = exp.predict_model(list_models[1])
    plot_parameter_validation_curve(xtrain.drop('shot_made_flag', axis=1), ytrain, 'C', {'C': [0.001, 0.01, 0.1, 1, 10]},
                                        list_models[1], 'Regressão Logística', 'f1',
                                        logx=True)
    
    plt.savefig('lr_validation_curve.png')
    mlflow.log_artifact('lr_validation_curve.png')
    #os.remove('dt_validation_curve.png')

    mlflow.log_metrics({
        'lr_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'lr_f1': f1_score(yhat_test.target, yhat_test.prediction_label),
    })


    # FINALIZACAO MELHOR MODELO

    tune_model = exp.tune_model(list_models[0],
                                optimize = 'f1',
                                search_algorithm = 'random',
                                n_iter = 4)
    yhat_test = exp.predict_model(tune_model, raw_score=True)
    mlflow.log_metrics({
        'final_model_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'final_model_f1': f1_score(yhat_test.target, yhat_test.prediction_label),
    })
    
    yhat_test.to_parquet('prediction_test.parquet')
    mlflow.log_artifact('prediction_test.parquet')
    final_model = exp.finalize_model(tune_model)

    # EXPORTACAO PARA LOG E REGISTRO DO MODELO
    exp.save_model(final_model, f'./{registered_model_name}')
    # Carrega novamente o pipeline + bestmodel
    model_pipe = exp.load_model(f'./{registered_model_name}')
    # Assinatura do Modelo Inferida pelo MLFlow
    model_features = list(xtrain.drop('shot_made_flag', axis=1).columns)
    inf_signature = infer_signature(xtrain[model_features],
                                    model_pipe.predict_proba(xtrain.drop('shot_made_flag', axis=1)))
    # Exemplo de entrada para o MLmodel
    input_example = {x: xtrain[x].values[:nexamples] for x in model_features}
    # Log do pipeline de modelagem do sklearn e registrar como uma nova versao
    mlflow.sklearn.log_model(
        sk_model=model_pipe,
        artifact_path="sklearn-model",
        registered_model_name=registered_model_name,
        signature = inf_signature,
        input_example = input_example,
        pyfunc_predict_fn='predict_proba'
    )
     # Criacao do cliente do servico MLFlow e atualizacao versao modelo
    client = MlflowClient()
    if model_version == -1:
        model_version = client.get_latest_versions(registered_model_name)[-1].version
    # Registrar o modelo como staging
    client.set_registered_model_alias(
        name    = registered_model_name,
        alias   = "staging",
        version = model_version
    )


In [13]:
!pip show joblib

Name: joblib
Version: 1.4.0
Summary: Lightweight pipelining with Python functions
Home-page: 
Author: 
Author-email: Gael Varoquaux <gael.varoquaux@normalesup.org>
License: BSD 3-Clause
Location: /opt/anaconda3/lib/python3.9/site-packages
Requires: 
Required-by: imbalanced-learn, nltk, pmdarima, pycaret, pyod, scikit-learn, scikit-plot


In [14]:
!pip install --upgrade joblib

In [18]:
!pip install "pycaret[analysis]"
!pip install "pycaret[models]"
!pip install "pycaret[tuner]"
!pip install "pycaret[mlops]"
!pip install "pycaret[parallel]"
!pip install "pycaret[test]"

  Using cached shap-0.45.0-cp39-cp39-macosx_10_9_x86_64.whl.metadata (24 kB)
  Using cached interpret-0.6.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached umap_learn-0.5.6-py3-none-any.whl.metadata (21 kB)
  Using cached PyYAML-5.3.1-cp39-cp39-macosx_10_9_x86_64.whl
  Using cached ydata_profiling-4.7.0-py2.py3-none-any.whl.metadata (20 kB)
  Using cached explainerdashboard-0.4.7-py3-none-any.whl.metadata (3.8 kB)
  Using cached fairlearn-0.7.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached dash_auth-2.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached dash_bootstrap_components-1.6.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached dtreeviz-2.2.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached flask_simplelogin-0.1.2-py3-none-any.whl.metadata (3.4 kB)
  Using cached flask_wtf-1.2.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached jupyter_dash-0.4.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached oyaml-1.0-py2

In [16]:
# DIGITADO AQUI
import os
from sklearn.metrics import log_loss, f1_score
import mlflow
from pycaret.classification import *

registered_model_name = 'modelo_kobe'
nexamples = 5
model_version = -1
experiment_id = mlflow.create_experiment('Treinamento Kobe Atualizado')

with mlflow.start_run(experiment_id=experiment_id, run_name = 'Treinamento'):

    exp = pc.setup(
            data = xtrain,
            target = 'shot_made_flag',
            test_data = xtest,
            normalize=True,
            log_experiment = False
    )
    list_models = exp.compare_models(['lr', 'dt'], n_select=2, sort='f1')

    #ARVORE

    #exp.plot_model(list_models[0], plot='vc', save = True)
    yhat_test = exp.predict_model(list_models[0])
    plot_parameter_validation_curve(xtrain.drop('shot_made_flag', axis=1), ytrain, 'max_depth', {'max_depth': [2, 3, 4, 5, 6, 7, 8]},
                                        list_models[0], 'Árvore de Decisão', 'f1',
                                        logx=False)
    plt.savefig('dt_validation_curve.png')
    mlflow.log_artifact('dt_validation_curve.png')
    #os.remove('dt_validation_curve.png')

    mlflow.log_metrics({
        'dt_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'dt_f1': f1_score(yhat_test.target, yhat_test.prediction_label),
    })

    #REGRESSAO

    yhat_test = exp.predict_model(list_models[1])
    plot_parameter_validation_curve(xtrain.drop('shot_made_flag', axis=1), ytrain, 'C', {'C': [0.001, 0.01, 0.1, 1, 10]},
                                        list_models[1], 'Regressão Logística', 'f1',
                                        logx=True)
    
    plt.savefig('lr_validation_curve.png')
    mlflow.log_artifact('lr_validation_curve.png')
    #os.remove('dt_validation_curve.png')

    mlflow.log_metrics({
        'lr_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'lr_f1': f1_score(yhat_test.target, yhat_test.prediction_label),
    })


    # FINALIZACAO MELHOR MODELO

    tune_model = exp.tune_model(list_models[0],
                                optimize = 'f1',
                                search_algorithm = 'random',
                                n_iter = 4)
    yhat_test = exp.predict_model(tune_model, raw_score=True)
    mlflow.log_metrics({
        'final_model_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'final_model_f1': f1_score(yhat_test.target, yhat_test.prediction_label),
    })
    
    yhat_test.to_parquet('prediction_test.parquet')
    mlflow.log_artifact('prediction_test.parquet')
    final_model = exp.finalize_model(tune_model)

    # EXPORTACAO PARA LOG E REGISTRO DO MODELO
    exp.save_model(final_model, f'./{registered_model_name}')
    # Carrega novamente o pipeline + bestmodel
    model_pipe = exp.load_model(f'./{registered_model_name}')
    # Assinatura do Modelo Inferida pelo MLFlow
    model_features = list(xtrain.drop('shot_made_flag', axis=1).columns)
    inf_signature = infer_signature(xtrain[model_features],
                                    model_pipe.predict_proba(xtrain.drop('shot_made_flag', axis=1)))
    # Exemplo de entrada para o MLmodel
    input_example = {x: xtrain[x].values[:nexamples] for x in model_features}
    # Log do pipeline de modelagem do sklearn e registrar como uma nova versao
    mlflow.sklearn.log_model(
        sk_model=model_pipe,
        artifact_path="sklearn-model",
        registered_model_name=registered_model_name,
        signature = inf_signature,
        input_example = input_example,
        pyfunc_predict_fn='predict_proba'
    )
     # Criacao do cliente do servico MLFlow e atualizacao versao modelo
    client = MlflowClient()
    if model_version == -1:
        model_version = client.get_latest_versions(registered_model_name)[-1].version
    # Registrar o modelo como staging
    client.set_registered_model_alias(
        name    = registered_model_name,
        alias   = "staging",
        version = model_version
    )


ImportError: cannot import name '_format_load_msg' from 'joblib.memory' (/opt/anaconda3/lib/python3.9/site-packages/joblib/memory.py)

In [19]:
from pycaret.classification import setup, compare_models, create_model, evaluate_model, tune_model
# Importação básica para iniciar e configurar sessões PyCaret
from pycaret.classification import setup, compare_models, create_model, evaluate_model, tune_model

# Você pode precisar de mais funções específicas dependendo do que você quer fazer
from pycaret.regression import predict_model
from pycaret.clustering import assign_model
from pycaret.anomaly import plot_model

# Importação de utilitários de MLflow se você estiver usando funcionalidades de MLOps
import mlflow

# Checagem das versões das bibliotecas instaladas para garantir que tudo está conforme esperado
import pycaret
import sklearn
import joblib

print("Versão do PyCaret:", pycaret.__version__)
print("Versão do scikit-learn:", sklearn.__version__)
print("Versão do joblib:", joblib.__version__)


ImportError: cannot import name '_format_load_msg' from 'joblib.memory' (/opt/anaconda3/lib/python3.9/site-packages/joblib/memory.py)

In [ ]:
exp.plot_model?

In [20]:
import os
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, f1_score
import mlflow
from mlflow.tracking import MlflowClient

# Supondo que 'pc' é uma biblioteca importada corretamente
# Supondo que 'xtrain' e 'xtest' são os dados de treinamento e teste já definidos

registered_model_name = 'modelo_kobe'
nexamples = 5
model_version = -1

with mlflow.start_run(experiment_id=experiment_id, run_name='Treinamento'):
    exp = pc.setup(
        data=xtrain,
        target='shot_made_flag',
        test_data=xtest,
        normalize=True,
        log_experiment=False
    )

    # Comparando modelos
    list_models = exp.compare_models(include=['lr', 'dt'], n_select=2, sort='f1')
    exp.plot_model(list_models[0], plot='vc', save=True)
    yhat_test = exp.predict_model(list_models[0])

    # Log de métricas
    mlflow.log_metrics({
        'lr_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'lr_f1': f1_score(yhat_test.target, yhat_test.prediction_label)
    })

    plt.savefig('lr_validation_curve.png')
    mlflow.log_artifact('lr_validation_curve.png')
    # os.remove('lr_validation_curve.png')

    # Treinamento e previsão com o segundo modelo
    yhat_test = exp.predict_model(list_models[1])

    # Log de métricas para o segundo modelo
    mlflow.log_metrics({
        'dt_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'dt_f1': f1_score(yhat_test.target, yhat_test.prediction_label)
    })

    plt.savefig('dt_validation_curve.png')
    mlflow.log_artifact('dt_validation_curve.png')
    # os.remove('dt_validation_curve.png')

    # Ajuste de modelo
    tune_model = exp.tune_model(
        list_models[0],
        optimize='f1',
        search_algorithm='random',
        n_iter=4
    )

    yhat_test = exp.predict_model(tune_model)
    mlflow.log_metrics({
        'final_model_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'final_model_f1': f1_score(yhat_test.target, yhat_test.prediction_label)
    })

    final_model = exp.finalize_model(tune_model)

    # Exportação do modelo
    exp.save_model(final_model, f'./{registered_model_name}')
    model_pipe = exp.load_model(f'./{registered_model_name}')

    # Assinatura do modelo
    model_features = list(xtrain.drop('shot_made_flag', axis=1).columns)
    inf_signature = mlflow.models.infer_signature(xtrain[model_features], model_pipe.predict_proba(xtrain.drop('shot_made_flag', axis=1)))

    # Exemplo de entrada
    input_example = {x: xtrain[x].values[:nexamples] for x in model_features}

    # Log do modelo
    mlflow.sklearn.log_model(
        sk_model=model_pipe,
        artifact_path="sklearn-model",
        registered_model_name=registered_model_name,
        signature=inf_signature,
        input_example=input_example
    )

    # Gerenciamento de versões do modelo
    client = MlflowClient()
    if model_version == -1:
        model_version = client.get_latest_versions(registered_model_name, stages=['None'])[0].version
    client.set_registered_model_alias(
        name=registered_model_name,
        alias="staging",
        version=model_version
    )


NameError: name 'pc' is not defined

In [1]:
import pycaret.classification as pc

ImportError: cannot import name '_format_load_msg' from 'joblib.memory' (/opt/anaconda3/lib/python3.9/site-packages/joblib/memory.py)

In [20]:
import os
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, f1_score
import mlflow
from mlflow.tracking import MlflowClient

# Supondo que 'pc' é uma biblioteca importada corretamente
# Supondo que 'xtrain' e 'xtest' são os dados de treinamento e teste já definidos

registered_model_name = 'modelo_kobe'
nexamples = 5
model_version = -1

with mlflow.start_run(experiment_id=experiment_id, run_name='Treinamento'):
    exp = pc.setup(
        data=xtrain,
        target='shot_made_flag',
        test_data=xtest,
        normalize=True,
        log_experiment=False
    )

    # Comparando modelos
    list_models = exp.compare_models(include=['lr', 'dt'], n_select=2, sort='f1')
    exp.plot_model(list_models[0], plot='vc', save=True)
    yhat_test = exp.predict_model(list_models[0])

    # Log de métricas
    mlflow.log_metrics({
        'lr_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'lr_f1': f1_score(yhat_test.target, yhat_test.prediction_label)
    })

    plt.savefig('lr_validation_curve.png')
    mlflow.log_artifact('lr_validation_curve.png')
    # os.remove('lr_validation_curve.png')

    # Treinamento e previsão com o segundo modelo
    yhat_test = exp.predict_model(list_models[1])

    # Log de métricas para o segundo modelo
    mlflow.log_metrics({
        'dt_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'dt_f1': f1_score(yhat_test.target, yhat_test.prediction_label)
    })

    plt.savefig('dt_validation_curve.png')
    mlflow.log_artifact('dt_validation_curve.png')
    # os.remove('dt_validation_curve.png')

    # Ajuste de modelo
    tune_model = exp.tune_model(
        list_models[0],
        optimize='f1',
        search_algorithm='random',
        n_iter=4
    )

    yhat_test = exp.predict_model(tune_model)
    mlflow.log_metrics({
        'final_model_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'final_model_f1': f1_score(yhat_test.target, yhat_test.prediction_label)
    })

    final_model = exp.finalize_model(tune_model)

    # Exportação do modelo
    exp.save_model(final_model, f'./{registered_model_name}')
    model_pipe = exp.load_model(f'./{registered_model_name}')

    # Assinatura do modelo
    model_features = list(xtrain.drop('shot_made_flag', axis=1).columns)
    inf_signature = mlflow.models.infer_signature(xtrain[model_features], model_pipe.predict_proba(xtrain.drop('shot_made_flag', axis=1)))

    # Exemplo de entrada
    input_example = {x: xtrain[x].values[:nexamples] for x in model_features}

    # Log do modelo
    mlflow.sklearn.log_model(
        sk_model=model_pipe,
        artifact_path="sklearn-model",
        registered_model_name=registered_model_name,
        signature=inf_signature,
        input_example=input_example
    )

    # Gerenciamento de versões do modelo
    client = MlflowClient()
    if model_version == -1:
        model_version = client.get_latest_versions(registered_model_name, stages=['None'])[0].version
    client.set_registered_model_alias(
        name=registered_model_name,
        alias="staging",
        version=model_version
    )


NameError: name 'pc' is not defined

In [20]:
import os
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, f1_score
import mlflow
from mlflow.tracking import MlflowClient

# Supondo que 'pc' é uma biblioteca importada corretamente
# Supondo que 'xtrain' e 'xtest' são os dados de treinamento e teste já definidos

registered_model_name = 'modelo_kobe'
nexamples = 5
model_version = -1

with mlflow.start_run(experiment_id=experiment_id, run_name='Treinamento'):
    exp = pc.setup(
        data=xtrain,
        target='shot_made_flag',
        test_data=xtest,
        normalize=True,
        log_experiment=False
    )

    # Comparando modelos
    list_models = exp.compare_models(include=['lr', 'dt'], n_select=2, sort='f1')
    exp.plot_model(list_models[0], plot='vc', save=True)
    yhat_test = exp.predict_model(list_models[0])

    # Log de métricas
    mlflow.log_metrics({
        'lr_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'lr_f1': f1_score(yhat_test.target, yhat_test.prediction_label)
    })

    plt.savefig('lr_validation_curve.png')
    mlflow.log_artifact('lr_validation_curve.png')
    # os.remove('lr_validation_curve.png')

    # Treinamento e previsão com o segundo modelo
    yhat_test = exp.predict_model(list_models[1])

    # Log de métricas para o segundo modelo
    mlflow.log_metrics({
        'dt_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'dt_f1': f1_score(yhat_test.target, yhat_test.prediction_label)
    })

    plt.savefig('dt_validation_curve.png')
    mlflow.log_artifact('dt_validation_curve.png')
    # os.remove('dt_validation_curve.png')

    # Ajuste de modelo
    tune_model = exp.tune_model(
        list_models[0],
        optimize='f1',
        search_algorithm='random',
        n_iter=4
    )

    yhat_test = exp.predict_model(tune_model)
    mlflow.log_metrics({
        'final_model_log_loss': log_loss(yhat_test.target, yhat_test.prediction_label),
        'final_model_f1': f1_score(yhat_test.target, yhat_test.prediction_label)
    })

    final_model = exp.finalize_model(tune_model)

    # Exportação do modelo
    exp.save_model(final_model, f'./{registered_model_name}')
    model_pipe = exp.load_model(f'./{registered_model_name}')

    # Assinatura do modelo
    model_features = list(xtrain.drop('shot_made_flag', axis=1).columns)
    inf_signature = mlflow.models.infer_signature(xtrain[model_features], model_pipe.predict_proba(xtrain.drop('shot_made_flag', axis=1)))

    # Exemplo de entrada
    input_example = {x: xtrain[x].values[:nexamples] for x in model_features}

    # Log do modelo
    mlflow.sklearn.log_model(
        sk_model=model_pipe,
        artifact_path="sklearn-model",
        registered_model_name=registered_model_name,
        signature=inf_signature,
        input_example=input_example
    )

    # Gerenciamento de versões do modelo
    client = MlflowClient()
    if model_version == -1:
        model_version = client.get_latest_versions(registered_model_name, stages=['None'])[0].version
    client.set_registered_model_alias(
        name=registered_model_name,
        alias="staging",
        version=model_version
    )


NameError: name 'pc' is not defined

In [ ]:
pip install pycaret


In [ ]:
pip install --upgrade pycaret joblib


In [ ]:
import pycaret.classification as pc
